In [ ]:
# This cell contains the parameters. It should have the tag "parameters"
SRCDIR = "."
patch_size = 128 #options 256 or 128
overlap = 0 #options 0 or half patch size ?
nb_patches = 100 #options between 1 and 1000

In [ ]:
img0 = 'input_0.png'
img1 = 'input_1.png'
img2 = 'input_2.png'

In [ ]:
import os
import sys
sys.path.append(SRCDIR)

In [ ]:
from IPython.display import display, Image
import matplotlib
#matplotlib.use('Agg')
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import imageio
import plotly.express as px
import plotly.graph_objects as go
#import sys
#sys.path.append('src_updated')
import src_updated.forensic_similarity as forsim #forensic similarity tool
from src_updated.utils.blockimage import tile_image #function to tile image into blocks

In [ ]:
def output(img0,img1,img2, patch_size, overlap, nb_patches):
    
    ### READ IMAGES
    
    I0 = imageio.imread(img0)
    I1 = imageio.imread(img1)
    I2 = imageio.imread(img2)
    
    ### EXTRACT PATCHES
    overlap = int(overlap*patch_size)
    np.random.seed(0)
    #tiles and xy coordinates of each tile for image 0
    T0,xy0 = tile_image(I0,width=patch_size,height=patch_size, 
                x_overlap=overlap,y_overlap=overlap)
    #tiles and xy coordinates of each tile for image 1
    T1,xy1 = tile_image(I1,width=patch_size,height=patch_size,
                x_overlap=overlap,y_overlap=overlap)
    #tiles and xy coordinates of each tile for image 2
    T2,xy2 = tile_image(I2,width=patch_size,height=patch_size,
                x_overlap=overlap,y_overlap=overlap)
    
    
    if len(T0)*len(T1)>= nb_patches and len(T0)*len(T2) >= nb_patches:
        ### SELECT PATCHES
        #inds0 = np.random.randint(0,len(T0),size=nb_patches) #select random indices
        #inds1 = np.random.randint(0,len(T1),size=nb_patches) 
        #inds2 = np.random.randint(0,len(T2),size=nb_patches) 
        #X0 = T0[inds0] #vector of randomly selected image tiles
        #X1 = T1[inds1] #vector of randomly selected image tiles
        #X2 = T2[inds2] #vector of randomly selected image tiles
        inds_aux_0_1 = np.random.choice(range(len(T0)*len(T1)), nb_patches, replace=False) 
        inds01 = inds_aux_0_1 % len(T0) #[i%len(T0) for i in inds_aux_0_1]
        inds1 = inds_aux_0_1 // len(T0) #[i//len(T0) for i in inds_aux_0_1]

        inds_aux_0_2 = np.random.choice(range(len(T0)*len(T2)), nb_patches, replace=False) 
        inds02 = inds_aux_0_2 % len(T0) #[i%len(T0) for i in inds_aux_0_2]
        inds2 = inds_aux_0_2 // len(T0) #[i//len(T0) for i in inds_aux_0_2]

        X01 = T0[inds01] #vector of randomly selected image tiles
        X1 = T1[inds1] #vector of randomly selected image tiles
        X02 = T0[inds02]
        X2 = T2[inds2] #vector of randomly selected image tiles


        ### READ PRE-TRAINED WEIGHTS

        if patch_size == 256:
            f_weights = os.path.join(SRCDIR, 'src_updated/pretrained/cam_256/-30')
            #f_weights= 'src_updated/pretrained/cam_256/-30'
            #tf.print(f_weights)
        elif patch_size == 128:
            f_weights = os.path.join(SRCDIR, 'src_updated/pretrained/cam_128/-30')
            #f_weights = 'src_updated/pretrained/cam_128/-30'
        else:
            print('Incorrect patch size')

        ### COMPUTE FORENSIC SIMILARITY
        #sim1 = forsim.calculate_forensic_similarity(X0,X1,f_weights,patch_size)
        #sim2 = forsim.calculate_forensic_similarity(X0,X2,f_weights,patch_size)
        sim1 = forsim.calculate_forensic_similarity(X01,X1,f_weights,patch_size)
        sim2 = forsim.calculate_forensic_similarity(X02,X2,f_weights,patch_size)
    else:
        sim1, sim2 = [], []
    return sim1, sim2

In [ ]:
#| output: false
sim1, sim2 = output(img0, img1, img2, patch_size, overlap, nb_patches)

::: {.panel-tabset}
### Reference image

In [ ]:
display(Image(img0))

### Image 1

In [ ]:
display(Image(img1))

### Image 2

In [ ]:
display(Image(img2))

### Output

In [ ]:
if sim1 == [] or sim2 == []:
    print(f'The images are too small to perform {nb_patches} patch-to-patch comparisons.')

else:
    fig = go.Figure()
    fig.add_trace(go.Histogram(
        x=sim1,
        name='Reference Image - Image 1', # name used in legend and hover labels
        xbins=dict( # bins used for histogram
            start=0,
            end=1,
            size=0.02
        ),
        marker_color='#076377',#'#a00d40',
        opacity=0.5
    ))
    fig.add_trace(go.Histogram(
        x=sim2,
        name='Reference Image - Image 2',
        xbins=dict(
            start=0,
            end=1,
            size=0.02
        ),
        marker_color=' #8260a0',#'#076377',
        opacity=0.5
    ))

    fig.update_layout(
        title_text='Forensic similarity', # title of plot
        xaxis_title_text='Forensic similarity score', # xaxis label
        yaxis_title_text='Count', # yaxis label
        barmode='overlay'
    )

    fig.show()
    fig.write_image("output_0.png")

:::